# Imports

In [1]:
import torch
import numpy as np
import pytorch_lightning as pl
import torchvision.transforms as tf
from tqdm import tqdm
from predict import *
from HIST2ST import *
# from dataset import ViT_HER2ST, ViT_SKIN
from scipy.stats import pearsonr,spearmanr
from torch.utils.data import DataLoader
# from pytorch_lightning.loggers import TensorBoardLogger
# from copy import deepcopy as dcp
# from collections import defaultdict as dfd
# from sklearn.metrics import adjusted_rand_score as ari_score
# from sklearn.metrics.cluster import normalized_mutual_info_score as nmi_score
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # For debugging CUDA errors

2025-07-24 19:29:19.334844: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX512F, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[easydl] tensorflow not available!


# Data Loading

In [2]:
# Create dictionaries to convert between Ensembl gene IDs and HGNC symbols
conversion = pd.read_csv('../../../../tahsin/HEST/assets/gene_ids/hgnc_complete_set.txt', sep="\t").loc[:, ['ensembl_gene_id', 'symbol']]
convert_to_ens = conversion.set_index('symbol')['ensembl_gene_id'].to_dict()
convert_to_sym = conversion.set_index('ensembl_gene_id')['symbol'].to_dict()

In [3]:
import os

meta_hest_path = "/work/bose_lab/tahsin/data/HEST"

meta_df = pd.read_csv(os.path.join(meta_hest_path, "HEST_v1_1_0.csv"))
meta_df = meta_df[meta_df['species'] == 'Homo sapiens']
# meta_df = meta_df[meta_df['disease_comment'] != 'SARS-CoV-2 Asymptomatic']

# Filter the dataframe if you need
# meta_df = meta_df[(meta_df['organ'] == 'Heart') & (meta_df['species'] == 'Homo sapiens')]
# print(meta_df.shape)

# Separate dataframe into training and validation sets
# meta_df = meta_df.sample(frac=1, random_state=42)  # Shuffle the dataframe
# train_frac = 0.8
# train_size = int(len(meta_df) * train_frac)
# train_df_80 = meta_df.iloc[:train_size]
# val_df_20 = meta_df.iloc[train_size:]

# ids = meta_df['id']

In [4]:
hest_path = "/work/bose_lab/tahsin/HEST/data/hest_human_only_jul11/patched_adata"
samples = os.listdir(hest_path)
samples = [sample for sample in samples if sample.endswith("preprocessed.h5ad")]
ids = [sample.split('_')[0] for sample in samples]

In [5]:
tag='5-7-2-8-4-16-32'
lr = 1e-5
k,p,d1,d2,d3,h,c=map(lambda x:int(x),tag.split('-'))
seed = 42
max_epochs = 350
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
from sklearn.model_selection import train_test_split

# Split the ids into train and test sets (80% train, 20% test)
train_ids, test_ids = train_test_split(ids, test_size=0.2, random_state=seed)

print(f"Number of training samples: {len(train_ids)}")
print(f"Number of test samples: {len(test_ids)}")

Number of training samples: 519
Number of test samples: 130


# Hist2ST training

In [7]:
fold=5
neighbours = 5
# data='her2st'
data = 'hest1k'
if data == 'her2st':
    prune = 'Grid'
    genes = 785
elif data == 'cscc':
    prune = 'NA'
    genes = 171
elif data == 'hest1k':
    prune = 'NA'# IDK??
    genes = 1000

In [8]:
adata_path = os.path.join(hest_path, f"{ids[0]}_preprocessed.h5ad")
sample = ad.read_h5ad(adata_path)
print(sample)

AnnData object with n_obs × n_vars = 1282 × 25812
    obs: 'in_tissue', 'pxl_col_in_fullres', 'pxl_row_in_fullres', 'array_col', 'array_row', 'n_counts', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito', 'patches_dir', 'patch_available', 'n_patches', 'individual_patch_file', 'barcode_matching_deferred', 'sample_id', 'original_st_file'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mito', 'original_gene_names', 'gene_id', 'species', 'alias_type', 'original_name'
    uns: 'gene_filtering', 'gene_filtering_config', 'gene_reordering', 'gene_standardization', 'gene_standardization_config', 'log1p', 'spatial'
    obsm: 'spatial'


In [16]:
trainset = pk_load(fold,'train', sample_ids=train_ids,dataset =data, flatten=False, adj=True, ori=True, prune=prune, neighs=neighbours)
train_loader = DataLoader(trainset, batch_size=1, num_workers=0, shuffle=True)

testset = pk_load(fold,'test', sample_ids=test_ids,dataset =data, flatten=False, adj=True, ori=True, prune=prune, neighs=neighbours)
test_loader = DataLoader(testset, batch_size=1, num_workers=0, shuffle=False)


Loading dataset with parameters:
  Dataset: hest1k
  Mode: train
  Sample IDs: ['INT25', 'NCBI866', 'SPA35', 'SPA67', 'NCBI592', 'SPA56', 'NCBI474', 'NCBI491', 'NCBI763', 'SPA114', 'SPA85', 'MEND161', 'INT26', 'TENX50', 'NCBI483', 'NCBI765', 'MEND66', 'SPA33', 'SPA126', 'NCBI564', 'TENX114', 'TENX116', 'SPA152', 'SPA132', 'NCBI858', 'SPA96', 'NCBI634', 'MEND162', 'NCBI712', 'INT8', 'INT20', 'SPA83', 'MISC41', 'SPA115', 'MISC134', 'MISC139', 'MISC60', 'NCBI635', 'NCBI517', 'MISC50', 'NCBI864', 'TENX16', 'NCBI813', 'NCBI593', 'MEND18', 'SPA74', 'NCBI461', 'NCBI534', 'NCBI478', 'MISC108', 'MEND90', 'SPA31', 'MEND34', 'MISC109', 'NCBI881', 'SPA110', 'INT24', 'MEND93', 'SPA113', 'SPA101', 'SPA36', 'SPA58', 'TENX90', 'SPA52', 'SPA97', 'NCBI518', 'SPA102', 'NCBI875', 'NCBI468', 'SPA81', 'MEND157', 'MISC23', 'NCBI515', 'MISC119', 'SPA53', 'NCBI768', 'SPA104', 'TENX65', 'SPA76', 'MISC114', 'NCBI572', 'NCBI683', 'NCBI476', 'TENX156', 'TENX14', 'NCBI814', 'ZEN47', 'SPA69', 'SPA154', 'MISC71', 'M

In [10]:
from pytorch_lightning.loggers import TensorBoardLogger

log_name = 'hist2st-1_hestpreprocessed'
logger = TensorBoardLogger(
    'logs', 
    name=log_name
)

model=Hist2ST(
    depth1=d1, depth2=d2,depth3=d3,n_genes=genes,
    learning_rate=lr, kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5
)

In [15]:
from datetime import date
today = date.today()
print(f"Today's date: {today}")
logger=None
trainer = pl.Trainer( accelerator = 'gpu', 
                     max_epochs=max_epochs,
                     logger=logger,
                     check_val_every_n_epoch=2,
)
trainer.fit(model, train_loader, test_loader)

import os
from datetime import date
if not os.path.isdir("./model/"):
    os.mkdir("./model/")

torch.save(model.state_dict(),f"./model/Hist2ST_{today}.ckpt")

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.


Today's date: 2025-07-24


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type       | Params | Mode 
-------------------------------------------------------
0 | patch_embedding | Conv2d     | 4.7 K  | train
1 | x_embed         | Embedding  | 65.5 K | train
2 | y_embed         | Embedding  | 65.5 K | train
3 | vit             | ViT        | 71.4 M | train
4 | mean            | Sequential | 1.0 M  | train
5 | disp            | Sequential | 1.0 M  | train
6 | pi              | Sequential | 1.0 M  | train
7 | coef            | Sequential | 1.1 M  | train
8 | gene_head       | Sequential | 1.0 M  | train
-------------------------------------------------------
76.7 M    Trainable params
0         Non-trainable params
76.7 M    Total params
306.685   Total estimated model params size (MB)
203       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                     | 0/? [00:00<?, ?it/s]


Loading sample MISC30
AnnData object:
  obs_names unique: True
  var_names unique: True

Processing sample MISC30
Sample MISC30 has 0 common genes with the dataset


ValueError: Found array with 0 feature(s) (shape=(2729, 0)) while a minimum of 1 is required by the normalize function.


# Hist2ST Prediction
To run the trained model, please select the trained model and replace the value of the variable fold with the number in the name of the selected trained model.

In [ ]:
pred, gt = test(model, test_loader,'cuda')
R=get_R(pred,gt)[0]
print('Pearson Correlation:',np.nanmean(R))
# clus,ARI=cluster(pred,label)
# print('ARI:',ARI)

# Validation

In [ ]:
fold=5
# data='her2st'
data = 'hest1k'
if data == 'her2st':
    prune = 'Grid'
    genes = 785
elif data == 'cscc':
    prune = 'NA'
    genes = 171
elif data == 'hest1k':
    prune = 'NA'# IDK??
    genes = 1000

In [ ]:

model=Hist2ST(
    depth1=d1, depth2=d2,depth3=d3,n_genes=genes, 
    kernel_size=k, patch_size=p,
    heads=h, channel=c, dropout=0.2,
    zinb=0.25, nb=False,
    bake=5, lamb=0.5, 
)
model.load_state_dict(torch.load(f'./downloads/{fold}-Hist2ST.ckpt'))
print(f"X embedding dimension: {model.x_embed.num_embeddings}")
print(f"Y embedding dimension: {model.y_embed.num_embeddings}")
sample_data = trainset[0]
# patches, positions, expression, adj, centers = sample_data
# print(f"Position coordinate range: {positions.min()} to {positions.max()}")



Loading dataset with parameters:
  Dataset: her2st
  Mode: test
  Sample IDs: None
  Fold: 5
['B1']
Loading imgs...
Loading metadata...
X embedding dimension: 64
Y embedding dimension: 64


In [ ]:
# Your current parameters from tag '5-7-2-8-4-16-32'
print(f"Current params: k={k}, p={p}, d1={d1}, d2={d2}, d3={d3}, h={h}, c={c}")
print(f"patch_size={p}, channel={c}")

# Check what the model actually expects vs. receives
sample_data = testset[0]
# patches, positions, expression, adj, centers = sample_data
# print(f"Input patch shape: {patches.shape}")
# print(f"Model expects channel dimension: {c}")

Current params: k=5, p=7, d1=2, d2=8, d3=4, h=16, c=32
patch_size=7, channel=32


In [ ]:
# Load and inspect the checkpoint
checkpoint = torch.load(f'./downloads/{fold}-Hist2ST.ckpt')

print("Checkpoint keys (first 20):")
for i, key in enumerate(checkpoint.keys()):
    print(f"{i+1:2d}: {key}")
    if i >= 19:
        break
print(f"... total keys: {len(checkpoint.keys())}")

# If the checkpoint contains metadata about architecture
if 'hyper_parameters' in checkpoint:
    print("\nSaved hyperparameters:")
    for k, v in checkpoint['hyper_parameters'].items():
        print(f"  {k}: {v}")

Checkpoint keys (first 20):
 1: patch_embedding.weight
 2: patch_embedding.bias
 3: x_embed.weight
 4: y_embed.weight
 5: vit.transformer.layer1.0.dw.0.weight
 6: vit.transformer.layer1.0.dw.0.bias
 7: vit.transformer.layer1.0.dw.1.weight
 8: vit.transformer.layer1.0.dw.1.bias
 9: vit.transformer.layer1.0.dw.1.running_mean
10: vit.transformer.layer1.0.dw.1.running_var
11: vit.transformer.layer1.0.dw.1.num_batches_tracked
12: vit.transformer.layer1.0.dw.3.weight
13: vit.transformer.layer1.0.dw.3.bias
14: vit.transformer.layer1.0.dw.4.weight
15: vit.transformer.layer1.0.dw.4.bias
16: vit.transformer.layer1.0.dw.4.running_mean
17: vit.transformer.layer1.0.dw.4.running_var
18: vit.transformer.layer1.0.dw.4.num_batches_tracked
19: vit.transformer.layer1.0.pw.0.weight
20: vit.transformer.layer1.0.pw.0.bias
... total keys: 162


In [ ]:
test_set = pk_load(2, 'test', dataset=data, flatten=False, adj=True, ori=True, prune=prune)
test2_loader = DataLoader(test_set, batch_size=1, num_workers=0, shuffle=False)


Loading dataset with parameters:
  Dataset: her2st
  Mode: test
  Sample IDs: None
  Fold: 2
['A4']
Loading imgs...
Loading metadata...


In [ ]:
for d in iter(test2_loader):
    for k in d:
        print(f"Processing sample with label: {k.shape}")
    print("=====")

Processing sample with label: torch.Size([1, 343, 3, 112, 112])
Processing sample with label: torch.Size([1, 343, 2])
Processing sample with label: torch.Size([1, 343, 785])
Processing sample with label: torch.Size([1, 343, 343])
Processing sample with label: torch.Size([1, 343, 785])
Processing sample with label: torch.Size([1, 343])
Processing sample with label: torch.Size([1, 343, 2])
=====


In [ ]:
for d in iter(test_loader):
    for k in d:
        print(f"Processing sample with label: {k.shape}")
    print("=====")

Processing sample with label: torch.Size([1, 295, 3, 112, 112])
Processing sample with label: torch.Size([1, 295, 2])
Processing sample with label: torch.Size([1, 295, 785])
Processing sample with label: torch.Size([1, 295, 295])
Processing sample with label: torch.Size([1, 295, 785])
Processing sample with label: torch.Size([1, 295])
Processing sample with label: torch.Size([1, 295, 2])
=====


In [ ]:
all_ids = meta_df['id'].tolist()
np.random.seed(42)
np.random.shuffle(all_ids)

split_idx = int(len(all_ids) * 0.8)
sample_ids = all_ids[split_idx:]

In [ ]:
# for sample_id in sample_ids:
#     hestset = pk_load(fold, 'test', sample_ids=[sample_id], dataset='hest1k', flatten=False, adj=True, ori=True, prune=prune)
#     hest_loader = DataLoader(hestset, batch_size=1, num_workers=0, shuffle=False)
#     for d in iter(hest_loader):
#         for k in d:
#             print(f"Processing HEST sample with label: {k.shape}")
#         print("=====")

In [ ]:
# meta_df

In [ ]:
def get_R_robust(data1, data2):
    """Robust correlation calculation handling edge cases"""
    x1 = data1.X
    x2 = data2.X
    
    print(f"Data shapes: pred {x1.shape}, gt {x2.shape}")
    
    if x1.shape != x2.shape:
        print("ERROR: Shape mismatch between predicted and ground truth data")
        return np.array([]), np.array([])
    
    r_values = []
    
    for i in range(x1.shape[1]):  # For each gene
        col1 = x1[:, i]
        col2 = x2[:, i]
        
        # Check for constant columns (zero variance)
        if np.var(col1) == 0 or np.var(col2) == 0:
            r_values.append(0.0)
            continue
            
        # Check for valid data points
        mask = np.isfinite(col1) & np.isfinite(col2)
        if mask.sum() < 2:
            r_values.append(0.0)
            continue
            
        try:
            r, _ = pearsonr(col1[mask], col2[mask])
            r_values.append(r if np.isfinite(r) else 0.0)
        except:
            r_values.append(0.0)
    
    return np.array(r_values), np.array([])



In [ ]:
from collections import Counter
import anndata as ad

sample_id = 'MEND24'
adata_path = os.path.join(hest_path, "st", f"{sample_id}.h5ad")
adata = ad.read_h5ad(adata_path)
# names = adata.obs_names.tolist()
# Find and print duplicate names in adata.obs_names

print(adata)

AnnData object with n_obs × n_vars = 136 × 26437
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mito'
    uns: 'spatial'
    obsm: 'spatial'


In [ ]:
print(adata.obsm['spatial'])

[[16655  5937]
 [15997  6668]
 [16653  6655]
 [15254  7371]
 [15989  7372]
 [16687  7376]
 [15262  8094]
 [15998  8105]
 [16665  8097]
 [15286  8838]
 [15997  8835]
 [16680  8813]
 [15301  9557]
 [16018  9553]
 [16682  9562]
 [15321 10290]
 [16021 10286]
 [16686 10281]
 [15330 11015]
 [16024 10997]
 [16670 10997]
 [15310 11783]
 [16038 11742]
 [16678 11723]
 [15326 12465]
 [16045 12474]
 [16724 12462]
 [15338 13199]
 [16049 13176]
 [16718 13188]
 [15307 13925]
 [16061 14008]
 [16735 13926]
 [15319 14642]
 [16038 14586]
 [16730 14653]
 [15347 15375]
 [16059 15437]
 [16739 15379]
 [15348 16081]
 [16072 16106]
 [16754 16094]
 [15360 16830]
 [16108 16836]
 [16759 16836]
 [15371 17551]
 [16093 17560]
 [16769 17572]
 [15411 18284]
 [16109 18324]
 [16768 18302]
 [15400 19007]
 [16130 19037]
 [15406 19743]
 [16118 19743]
 [15393 20476]
 [16120 20491]
 [16124 21218]
 [16119 21942]
 [25420  6513]
 [24675  7303]
 [25430  7251]
 [26125  7248]
 [26870  7252]
 [24678  8037]
 [25443  7974]
 [26146  7

In [ ]:
for sample_id in sample_ids:
# for sample_id in ['MISC44']:
    # Check dataset before testing
    hestset = pk_load(fold, 'test', sample_ids=[sample_id], dataset='hest1k', flatten=False, adj=True, ori=True, prune=prune)
    # Debug dataset contents
    print("Dataset info:")
    print(f"Dataset length: {len(hestset)}")
    if hasattr(hestset, 'names'):
        print(f"Sample names: {hestset.names}")
        
    # Check first item
    if len(hestset) <1:
        print("Warning: Dataset is empty. Skipping.")
        continue
    elif len(hestset) == 1:
        sample = hestset
    else: sample = hestset[0]
    print("\nFirst sample:")
    # Safely inspect sample contents
    if isinstance(sample, tuple) or isinstance(sample, list):
        # If sample is a collection of items (like tuple from dataset __getitem__)
        for i, item in enumerate(sample):
            if isinstance(item, torch.Tensor):
                print(f"Item {i}: Tensor shape {item.shape}, dtype {item.dtype}")
            elif isinstance(item, np.ndarray):
                print(f"Item {i}: Array shape {item.shape}, dtype {item.dtype}")
            else:
                print(f"Item {i}: Type {type(item)}")
    elif isinstance(sample, np.ndarray):
        # If sample is a single numpy array
        print(f"Sample is ndarray with shape {sample.shape}")
    else:
        # For other types
        print(f"Sample type: {type(sample)}")

    # Create dataloader and check
    hest_loader = DataLoader(hestset, batch_size=1, num_workers=0, shuffle=False)
    print(f"\nDataloader batches: {len(hest_loader)}")


Loading dataset with parameters:
  Dataset: hest1k
  Mode: test
  Sample IDs: ['NCBI474']
  Fold: 5
HEST Dataset: 1 samples (test)
['NCBI474']
Using 785 genes

Dataset loaded:
  Length: 1
Dataset info:
Dataset length: 1
Sample names: ['NCBI474']

First sample:
Sample type: <class 'dataset.ViT_HEST1K'>

Dataloader batches: 1

Loading dataset with parameters:
  Dataset: hest1k
  Mode: test
  Sample IDs: ['NCBI539']
  Fold: 5
HEST Dataset: 1 samples (test)
['NCBI539']
Using 785 genes

Dataset loaded:
  Length: 1
Dataset info:
Dataset length: 1
Sample names: ['NCBI539']

First sample:
Sample type: <class 'dataset.ViT_HEST1K'>

Dataloader batches: 1

Loading dataset with parameters:
  Dataset: hest1k
  Mode: test
  Sample IDs: ['MISC61']
  Fold: 5
HEST Dataset: 1 samples (test)
['MISC61']
Using 785 genes

Dataset loaded:
  Length: 1
Dataset info:
Dataset length: 1
Sample names: ['MISC61']

First sample:
Sample type: <class 'dataset.ViT_HEST1K'>

Dataloader batches: 1

Loading dataset with 

In [ ]:
correlations = []
for sample_id in sample_ids:
# for sample_id in ['SPA102']:
    hestset = pk_load(fold, 'test', sample_ids=[sample_id], dataset='hest1k', flatten=False, adj=True, ori=True, prune=prune)
    hest_loader = DataLoader(hestset, batch_size=1, num_workers=0, shuffle=False)
    pred, gt = test(model, hest_loader,'cuda')
    # Use robust correlation
    R, _ = get_R_robust(pred, gt)
    print(f'Pearson Correlation: {np.nanmean(R):.4f}')
    print(f'Valid correlations: {np.sum(~np.isnan(R))}/{len(R)}')
    correlations.append([sample_id, np.nanmean(R)])


Loading dataset with parameters:
  Dataset: hest1k
  Mode: test
  Sample IDs: ['NCBI474']
  Fold: 5
HEST Dataset: 1 samples (test)
['NCBI474']
Using 785 genes

Dataset loaded:
  Length: 1
Number of batches in dataloader: 1


  0%|                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]


Loading sample NCBI474
AnnData object:
  obs_names unique: True
  var_names unique: True

Processing sample NCBI474
Sample NCBI474 has 758 common genes with the dataset
Batch 0: patch shape = torch.Size([1, 598, 3, 112, 112])


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.67s/it]


Pred index is unique: True
GT index is unique: True
Data shapes: pred (598, 785), gt (598, 785)
Pearson Correlation: 0.0000
Valid correlations: 785/785

Loading dataset with parameters:
  Dataset: hest1k
  Mode: test
  Sample IDs: ['NCBI539']
  Fold: 5
HEST Dataset: 1 samples (test)
['NCBI539']
Using 785 genes

Dataset loaded:
  Length: 1
Number of batches in dataloader: 1


...

  0%|                                                                                                                                                                            | 0/1 [00:00<?, ?it/s]


Loading sample MISC44
AnnData object:
  obs_names unique: True
  var_names unique: True

Processing sample MISC44
Sample MISC44 has 753 common genes with the dataset


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.34s/it]

Batch 0: patch shape = torch.Size([1, 932, 3, 112, 112])
Pred index is unique: True
GT index is unique: True
Data shapes: pred (932, 785), gt (932, 785)


Pearson Correlation: -0.0703
Valid correlations: 785/785


In [ ]:
correlations

[['NCBI474', np.float64(0.0)],
 ['NCBI539', np.float64(0.0)],
 ['MISC61', np.float64(0.05613562489895352)],
 ['SPA102', np.float64(0.0)],
 ['TENX50', np.float64(0.0)],
 ['MISC139', np.float64(0.0)],
 ['TENX24', np.float64(0.02715647069153014)],
 ['NCBI537', np.float64(0.026139554347261597)],
 ['NCBI566', np.float64(0.0)],
 ['NCBI463', np.float64(0.0)],
 ['MISC51', np.float64(0.02474570717591389)],
 ['SPA104', np.float64(0.0)],
 ['NCBI711', np.float64(0.0)],
 ['MISC29', np.float64(0.0)],
 ['SPA138', np.float32(0.109641716)],
 ['MEND51', np.float64(0.026139554347261597)],
 ['MEND41', np.float64(0.02385501627377264)],
 ['NCBI565', np.float64(0.0)],
 ['SPA127', np.float32(0.010961207)],
 ['TENX154', np.float64(0.070489573951775)],
 ['NCBI497', np.float64(0.0)],
 ['MISC48', np.float64(0.12837191588530067)],
 ['MEND24', np.float64(0.0)],
 ['NCBI636', np.float64(0.0)],
 ['MISC126', np.float64(0.0)],
 ['MISC142', np.float64(0.0)],
 ['SPA144', np.float64(0.36106996167996885)],
 ['TENX115', np.f

In [ ]:
import pandas as pd
import numpy as np

# Save correlations to CSV
correlations_df = pd.DataFrame(correlations, columns=['sample_id', 'pearson_correlation'])
correlations_df.to_csv('correlations.csv', index=False)

# Calculate the average correlation
average_corr = np.nanmean(correlations_df['pearson_correlation'])
print(f"Average Pearson correlation: {average_corr:.4f}")

Average Pearson correlation: 0.0156


In [ ]:
meta_df.columns

Index(['dataset_title', 'id', 'image_filename', 'organ', 'disease_state',
       'oncotree_code', 'species', 'patient', 'st_technology',
       'data_publication_date', 'license', 'study_link', 'download_page_link1',
       'inter_spot_dist', 'spot_diameter', 'spots_under_tissue',
       'preservation_method', 'nb_genes', 'treatment_comment',
       'pixel_size_um_embedded', 'pixel_size_um_estimated', 'magnification',
       'fullres_px_width', 'fullres_px_height', 'tissue', 'disease_comment',
       'subseries', 'hest_version_added'],
      dtype='object')

In [ ]:
# meta_df[meta_df['id']=='NCBI573']
meta_df[meta_df['disease_comment'] == 'SARS-CoV-2 Asymptomatic']

,dataset_title,id,image_filename,organ,disease_state,oncotree_code,species,patient,st_technology,data_publication_date,...,treatment_comment,pixel_size_um_embedded,pixel_size_um_estimated,magnification,fullres_px_width,fullres_px_height,tissue,disease_comment,subseries,hest_version_added


In [ ]:
import anndata
ad1 = anndata.read_h5ad(os.path.join(hest_path, "st", "MISC139.h5ad"))
print(type(adz.var_names[0]))

AttributeError: module 'anndata' has no attribute 'var_names'

In [ ]:
# Clean the data by replacing NaN/Inf values
def clean_data(adata):
    """Remove NaN and Inf values from AnnData object"""
    X = adata.X.copy()
    # Replace NaN with 0 or mean
    X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
    # Or use median imputation
    # mask = np.isnan(X) | np.isinf(X)
    # X[mask] = np.nanmedian(X)
    adata_clean = anndata.AnnData(X)
    adata_clean.obsm = adata.obsm.copy()
    return adata_clean

# Clean both datasets before correlation
pred_clean = clean_data(pred)

R = get_R(pred_clean, gt)[0]
print('Pearson Correlation:', np.nanmean(R))

AttributeError: 'AnnData' object has no attribute 'AnnData'

In [ ]:
# Before calling get_R, inspect your data
print("Checking pred data:")
print(f"  Shape: {pred.X.shape}")
print(f"  Has NaN: {np.isnan(pred.X).any()}")
print(f"  Has Inf: {np.isinf(pred.X).any()}")
print(f"  Min value: {np.nanmin(pred.X)}")
print(f"  Max value: {np.nanmax(pred.X)}")

print("Checking gt data:")
print(f"  Shape: {gt.X.shape}")
print(f"  Has NaN: {np.isnan(gt.X).any()}")
print(f"  Has Inf: {np.isinf(gt.X).any()}")
print(f"  Min value: {np.nanmin(gt.X)}")
print(f"  Max value: {np.nanmax(gt.X)}")

In [ ]:
R=get_R(pred,gt)[0]
print('Pearson Correlation:',np.nanmean(R))


# clus,ARI=cluster(pred,label)
# print('ARI:',ARI)
